In [1]:
import os
import tensorflow as tf
from PIL import Image
import argparse
import cv2
import tqdm
import hashlib
from object_detection.utils import dataset_util
from collections import defaultdict
from random import shuffle

//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1)

In [87]:
import yaml
import imageio
def get_label(input_yaml):
    with open(input_yaml,'rb')as handle:
        images=yaml.load(handle)
    for i in range(len(images)):
        images[i]['path'] = os.path.abspath(os.path.join('./images',images[i]['path']))
            #if annotation is wrong where xmin > xmax or ymin > ymax fix
        for j, box in enumerate(images[i]['boxes']):
            if box['x_min'] > box['x_max']:
                images[i]['boxes'][j]['x_min'], images[i]['boxes'][j]['x_max'] = (
                images[i]['boxes'][j]['x_max'], images[i]['boxes'][j]['x_min'])
            if box['y_min'] > box['y_max']:
                images[i]['boxes'][j]['y_min'], images[i]['boxes'][j]['y_max'] = (
                images[i]['boxes'][j]['y_max'], images[i]['boxes'][j]['y_min'])
                
            #if annotation is outside of the img size, fix
        for j, box in enumerate(images[i]['boxes']):
            images[i]['boxes'][j]['x_max'] = min(box['x_max'],1279)
            images[i]['boxes'][j]['y_max'] = min(box['y_max'],1279)
            images[i]['boxes'][j]['x_min'] = max(box['x_min'],0)
            images[i]['boxes'][j]['y_min'] = max(box['y_min'],0)
                
            
    return images

In [88]:
def list_dict_to_dict_list(list_of_dicts):
    assert isinstance(list_of_dicts, list)
    dict_lists = defaultdict(list)
    for some_dict in list_of_dicts:
        for key, value in some_dict.items():
            dict_lists[key].append(value)
    return dict_lists

In [89]:
SIMPLIFIED_CLASSES = {
    'Green': 'Green',
    'GreenLeft': 'Green',
    'GreenRight': 'Green',
    'GreenStraight': 'Green',
    'GreenStraightRight': 'Green',
    'GreenStraightLeft': 'Green',
    'Yellow': 'Yellow',
    'Red': 'Red',
    'RedLeft': 'Red',
    'RedRight': 'Red',
    'RedStraight': 'Red',
    'RedStraightLeft': 'Red',
    'Off': 'Off',
    'off': 'Off',
}

In [90]:
def simplify_class(label_string):
    return SIMPLIFIED_CLASSES[label_string].encode('utf8')

In [1]:
TL_TABLE={
    'Off':4,
    'Green':1,
    'Yellow':2,
    'Red':3
}

In [92]:
def label_id(label_string):
    return TL_TABLE[SIMPLIFIED_CLASSES[label_string]]

In [ ]:
def clip(value):
    return max(0,min(value,1))

In [2]:
def to_tfrecord(labels,tfrecord_path):
    writer = tf.python_io.TFRecordWriter(tfrecord_path)

    for label in tqdm.tqdm(labels,desc='Creating tf_record'):
        img_path = label['path']
        image = cv2.imread(img_path)
        if image is None:
            print('Image?')
            raise IOError('Missing:{}'.format(img_path))
        height,width,_ = image.shape
        boxes = list_dict_to_dict_list(label['boxes'])
        classes = boxes['label']
        if not classes:
            continue 
        xmax = list(map(lambda x:clip(x/float(width)),boxes['x_max']))
        xmin = list(map(lambda x:clip(x/float(width)),boxes['x_min']))
        ymax = list(map(lambda y:clip(y/float(height)),boxes['y_max']))
        ymin = list(map(lambda y:clip(y/float(height)),boxes['y_min']))
        
        _, image = cv2.imencode('.png', image) 
        image = image.tostring()
        image_format = 'png'
        sha256=hashlib.sha256(image).hexdigest()
        complete_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(img_path.encode('utf8')),
            'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
            'image/format': dataset_util.bytes_feature(image_format.encode('utf8')),
            'image/source_id': dataset_util.bytes_feature(img_path.encode('utf8')),
            'image/key/sha256': dataset_util.bytes_feature(sha256.encode('utf8')),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
            'image/object/class/text': dataset_util.bytes_list_feature(list(map(simplify_class,classes))),
            'image/object/class/label': dataset_util.int64_list_feature(list(map(label_id,classes))),
        }))
        writer.write(complete_example.SerializeToString())

    writer.close()
            

In [95]:
def create_tfrecord(train_yaml,test_yaml,train_tfrecord_path,valid_tfrecord_path,test_tfrecord_path):
    train_labels = get_label(train_yaml)
    test_labels = get_label(test_yaml)
    #split train_labels to train and valid labels
    train_labels,valid_labels = train_labels[:int(len(train_labels)*5/6)],train_labels[int(len(train_labels)*5/6):]
    to_tfrecord(train_labels,train_tfrecord_path)
    to_tfrecord(valid_labels,valid_tfrecord_path)
    to_tfrecord(test_labels,test_tfrecord_path)

In [3]:
train_yaml = 'annotations/train.yaml'
test_yaml = 'annotations/test.yaml'
dataset_dir = './data/train'
train_tfrecord_path = './record_files/train.tfrecords'
test_tfrecord_path = './record_files/test.tfrecords'
valid_tfrecord_path = './record_files/valid.tfrecords'

In [96]:
create_tfrecord(train_yaml,test_yaml,train_tfrecord_path,valid_tfrecord_path,test_tfrecord_path)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """

Creating tf_record:  18%|█▊        | 776/4244 [00:31<01:38, 35.29it/s]


Creating tf_record:  35%|███▍      | 1475/4244 [01:03<02:44, 16.81it/s]


Creating tf_record:  51%|█████     | 2151/4244 [01:35<01:08, 30.65it/s]


Creating tf_record:  66%|██████▌   | 2799/4244 [02:08<01:19, 18.22it/s]


Creating tf_record:  81%|████████  | 3432/4244 [02:39<00:41, 19.40it/s]


...

Creating tf_record:  81%|████████  | 6732/8334 [06:01<01:39, 16.09it/s]


Creating tf_record:  86%|████████▌ | 7180/8334 [06:28<01:11, 16.15it/s]


Creating tf_record:  91%|█████████▏| 7616/8334 [06:54<00:44, 16.28it/s]


Creating tf_record:  97%|█████████▋| 8052/8334 [07:21<00:17, 16.29it/s]


Creating tf_record: 100%|██████████| 8334/8334 [07:32<00:00, 18.40it/s]

In [ ]:
for example in tf.python_io.tf_record_iterator(train_tfrecord_path):
    print(tf.train.Example.FromString(example))

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

